In [1]:
from Bio import SeqIO

In [56]:
for genes in SeqIO.parse('GRCh38_latest_genomic.fna', 'fasta'):
    print(genes.id)
    print(repr(genes.seq))
    print(len(genes))

NC_000001.11
Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN')
248956422
NT_187361.1
Seq('GAATTCAGCTGAGAAGAACAGGCAAGGACTTAGGAAATATTCCTTATTTGAAGG...TTC')
175055
NT_187362.1
Seq('AGGGGTCTGCTTAGAGAGGGTCTCATTAGTGGGGTCTAGTAGTGGGGTTTTGGT...TTC')
32032
NT_187363.1
Seq('gatcgagactatcctggctaacacggtgaaaccccgtctctactaaaaatacaa...ATC')
127682
NT_187364.1
Seq('gaattcattcgatgacgattccattcaattccgttcaatgattccattagattc...TTC')
66860
NT_187365.1
Seq('AAACTGtgcagcctctatggaaaacagtaccgtggttcctcaaaaaaattaaga...ATC')
40176
NT_187366.1
Seq('tgtgtatgtgtatgtgtatgtatgtgtatgtgtatgtgtatatgtatatgtata...GCT')
42210
NT_187367.1
Seq('GAATTCTCAGAACTTTCAAAACACAAATCTTCATCCGCAGGGATGTTCAGGAGG...TTC')
176043
NT_187368.1
Seq('GAGCATTCTCTAGTAGGAAGACCTTTTTAGAGTACAAAGGCCAGATCCTGCCCT...aac')
40745
NT_187369.1
Seq('CTAAACACAAAAagtttagctgggcatgatggtgcatgcctgtaatctgagcta...GGA')
41717
NC_000002.12
Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN')
242193529
NT_187370.1
Seq('GAATTCTCAGAAaccagtttgtga

In [57]:
# first burrows wheeler transform code attempt

import argparse
from operator import itemgetter

def bwt(s):
    n = len(s)
    m = sorted(s[i:n]+ s[0:i] for i in range(n))
    I = m.index(s)
    L = ''.join(q[-1] for q in m)
    return (I, L)


def ibwt(I, L):
    n = len(L)
    X = sorted([(i, x) for i, x in enumerate(L)], key=itemgetter(1))
    
    T = [None for i in range(n)]
    for i, y in enumerate(X):
        j, _ = y
        T[j] = i
        
        Tx = [I]
        for i in range(1, n):
            Tx.append(T[Tx[i-1]])
            
        S = [L[i] for i in Tx]
        S.reverse()
        return ''.join(S)

In [66]:
from Bio import SeqIO

with open('outfile.txt',"w") as f:
        for genes in SeqIO.parse('GRCh38_latest_genomic.fna', "fasta"):
                f.write(str(genes.id) + "\n")
                f.write(str(genes.seq[:10]) + "\n")  #first 10 base positions
                f.write(str(genes.seq[-10:]) + "\n") #last 10 base positions

In [69]:
# i think we would have to copy out the entire sequence for an individual gene for biopython to slice. it won't let me specifiy a slice for an specific gene

# this is an example of trying to extract a chromosome/piece of it
from Bio.Seq import Seq

seq = str(repr(genes.seq))
print(seq[6:12])

NNNNNN


In [14]:
# second bwt code attempt

def bwt(s):
    assert "\002" not in s and "\003" not in s, "Input string cannot contain STX and ETX characters"
    s = "\002" + s + "\003"  # Add start and end of text marker
    table = sorted(s[i:] + s[:i] for i in range(len(s)))  # Table of rotations of string
    last_column = [row[-1:] for row in table]  # Last characters of each row
    return "".join(last_column)  # Convert list of characters into string

def ibwt(r):
    """Apply inverse Burrows-Wheeler transform."""
    table = [""] * len(r)  # Make empty table
    for i in range(len(r)):
        table = sorted(r[i] + table[i] for i in range(len(r)))  # Add a column of r
    s = [row for row in table if row.endswith("\003")][0]  # Find the correct row (ending in ETX)
    return s.rstrip("\003").strip("\002")  # Get rid of start and end markers